In [0]:
# Read a single json file from a path
# Flatten the json file into dataframe
# Create a batch streaming auto loader bronze table using the dataframe

In [0]:
%sql
drop table workspace.training.song_details_brnz

In [0]:
# song_details_df = spark.read.format('json').option("inferSchema", True).option("rescuedDataColumn", '_rescued_data').option("multiline", True).load('/Volumes/workspace/training/training_volume/music-data/music stream 1.json')
from pyspark.sql.types import *

base_song_details_schema = StructType(
    [
        StructField("user_id", StringType(), False),
        StructField("artist_name", StringType(), False),
        StructField("song_title", StringType(), False),
        StructField("timestamp", DateType(), False),
        StructField("session_id", IntegerType(), False),
        StructField("device_type", StringType(), False),
    ]
)


song_details_df = (
    spark.readStream.format("json")
    .option("inferSchema", True)
    .option("rescuedDataColumn", "_rescued_data")
    .option("multiline", True)
    .option("schemaEvolution", True)
    .schema(base_song_details_schema)
    .load("/Volumes/workspace/training/training_volume/music-data/")
)


song_details_df.writeStream.format("delta").option("mergeSchema", True).option(
    "checkpointLocation", "/Volumes/workspace/training/training_volume/music-data/"
).trigger(once=True).toTable("workspace.training.song_details_brnz")

In [0]:
%sql
DESCRIBE HISTORY workspace.training.song_details_brnz

In [0]:
%sql
SELECT * FROM workspace.training.song_details_brnz